In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# Tarefa 4 - Decision Trees, Random Forest and K-Means
Fourth assessed coursework for the course: Técnicas e Algoritmos em Ciência de Dados

This tarefa provides an exciting opportunity for students to put their knowledge acquired in class into practice, using decision trees and random forests to solve a real-world problem in classification and delve into the world of unsupervised learning by implementing the K-means algorithm. Students will also get used to generating important plots during training to analyse the models' behaviour. 

## General guidelines:

* This work must be entirely original. You are allowed to research documentation for specific libraries, but copying solutions from the internet or your classmates is strictly prohibited. Any such actions will result in a deduction of points for the coursework.
* Before submitting your work, make sure to rename the file to the random number that you created for the previous coursework (for example, 289479.ipynb).
* Please try to not use any LLM-generated code. This coursework is designed for you to learn crucial concepts. Once you master them, then using LLMs become much easier.

## Notebook Overview:

1. [Decision Trees](#Decision_Trees) (30%)
2. [Random Forest](#Random_Forest) (30%)
3. [K-Means](#K-Means) (30%)

### Decision_Trees
## Part 1 - Decision Trees for Classification (value: 30%)

In this exercise, you will implement a decision tree for classifying whether the income of a person exceeds $50k/yr based on census data (adult_census_subset.csv in ECLASS). You will use the Information Gain based on the Gini Index as the impurity measure as the splitting criterion. The maximum depth and the minimum number of instances per leaf will be your stopping criteria. Be aware that some of the variables in this dataset are nominal (or categorical).

To complete this exercise, you will write code to build a decision tree for this problem: 

1. Dataset Splitting:
    - Load the provided dataset into your code.
	- Split the dataset into three sets: training, validation, and testing, with a 70/15/15 ratio, respectively. 
2. Implement a function to learn Decision Trees – the main conceptual steps are detailed below:
	- Initialize an empty decision tree.
	- Implement a recursive function to build the decision tree:
        - The stopping conditions for the recursive function are [note: satisfying only one of them is sufficient to stop the recursion]:
            - If the maximum depth is reached, stop growing the tree and create a leaf node with the frequency of the positive class for the remaining instances.
            - If the number of instances at a leaf node is less than the minimum number of instances per leaf, create a leaf node with the frequency of the positive class for those instances.
        - Your code will calculate the Information Gain (based on the Gini Index) for each possible value of each attribute and choose the attribute and value that maximizes the Information Gain (explanation below).
        - Your code will create a new internal node using the chosen attribute and value.
        - Your code will recursively call the build function on each subset of instances created by the split.
3. Implement a classification function. Implement a function to classify new instances using the decision tree:
	- For each instance, traverse the decision tree by comparing its attribute values to the decision nodes and move down the tree based on the attribute values until a leaf node is reached.
	- Return the frequency of the positive class that is associated with the leaf node as the prediction for the instance.
4. Run your algorithm and evaluate its performance:
	- Call the build function with the training set to construct the decision tree. You will vary the maximum depth and minimum number of instances per leaf to observe their effects on the decision tree performance. You will use the training set to learn the tree and the validation set using the Area Under the Roc Curve (AUROC) to find the optimal parameters. Try only shallow trees of a depth not deeper than 10, and min_instances not smaller than 10. If you try more extreme values, the training time could be too much.
	- Build a decision tree using the training + validation sets with the best combination of parameters.
	- Calculate the accuracy (threshold: 0.5) and AUROC of the decision tree in the testing set and report them.

To select the best split at each node you will use the Information Gain based on the Gini Index. The Gini Index measures the impurity of a node in a decision tree. To calculate the Information Gain based on the Gini Index, follow these steps [note: the same is explained in the slides for the case of entropy]:
- For each potential split (feature and value):
	- Calculate the Gini Index for node m (before any splits) using the class distribution within the node, using the following formula:
        - $G_m=\sum_{k=1}^K (\hat{p}_{mk} (1-\hat{p}_{mk})$, where $\hat{p}_{mk}$ represents the proportion of instances in the node $m$ that belong to class $k$.
	- Calculate the Gini Index for each possible outcome. This involves the following steps:
        - Split the data based on the attribute's possible outcomes.
        - Calculate the Gini index for each resulting subset using the same formula as in step a.
	- Calculate the weighted Gini index by summing up the Gini indexes of each subset, weighted by the proportion of instances it represents in the original node. The formula for the weighted Gini index ($W$) is as follows:
        - $W_V=\sum_v^V \frac{|S_v|}{|S|} G_{S_v} $ where $S_v$ is the node after the split and the sum iterates over all the children nodes; $|S_v|$ represents the cardinality of the node and $|S|$ the cardinality of the node before splitting; $G_{S_v}$ represents the Gini index of the node.
	- Calculate the information gain by subtracting the weighted Gini index obtained in step c. from the Gini index of the current node. The formula is as follows:
        - $InformationGain=G_{node}-W_V$


In [2]:
# Loading my dataset
adult_census = pd.read_csv("adult_census_subset.csv")

In [3]:
# Separating dataset and target
X = adult_census.drop("income", axis=1)
t = adult_census["income"]

In [4]:
from sklearn.datasets import load_iris

data = load_iris()
X = data.data
t = data.target

X = X[t != 2]
t = t[t != 2]

X = pd.DataFrame(X, columns=[i for i in range(X.shape[1])])
t = pd.Series(t)

In [5]:
# Separating my subsets
X_train, X_split, t_train, t_split = train_test_split(X, t, train_size=.7)
X_validation, X_test, t_validation, t_test = train_test_split(X_split, t_split, train_size=.5)

In [20]:
class DecisionTree:
    def __init__(self, max_depth=6, min_samples=3):
        self.max_depth = max_depth
        self.min_samples = min_samples
        self.root = None
    
    def fit(self, X: pd.DataFrame, t: pd.Series):
        """This function fits my decision tree on the dataset and target passed

        Parameters
        ----------
        X : pd.DataFrame
            The dataset that will be fitted onto the tree
        t : pd.Series
            The target for reach datapoint of the passed dataset
        """
        self.root = self.get_split(X, t, self.gini_index(t))
        self.root = self.recursive_growth(self.root, 0, X, t)
    
    def recursive_growth(self, node: dict, current_depth: int, X: pd.DataFrame, t: pd.Series):
        """
        Recursively grows a decision tree.
        
        Parameters
        ----------
        node : dict
            If the node is terminal, it contains only the "value" property, which determines the value
            to be used as a prediction.
            If the node is not terminal, the Node has the structure defined by `get_split`
        min_samples : int
            parameter for stopping criterion if a node has <= min_samples datapoints
        max_depth : int
            parameter for stopping criterion if a node belongs to this depth
        depth : int
            current distance from the root
        X : pd.DataFrame
            features (full dataset)
        t : pd.Series
            labels, targets (full dataset)
        
        Notes
        -----
        To create a terminal node, a dictionary is created with a single "value" key, with a value that
        is the mean of the target variable
        
        'left' and 'right' keys are added to non-terminal nodes, which contain (possibly terminal) nodes 
        from higher levels of the tree:
        'left' corresponds to the 'left_region' key, and 'right' to the 'right_region' key
        """
        l_idx = node['left_region']
        r_idx = node['right_region']

        if                                                                                  \
            (len(X) <= self.min_samples)             \
            or                                                                              \
            (current_depth >= self.max_depth)                                                    \
        :
            return {
                "value": np.mean(t)
            }
        
        # X = X.drop(node['feature_index'], axis=1)
        splited_infos = self.get_split(X, t, node['gini_index'])
        l_idx = splited_infos['left_region']
        r_idx = splited_infos['right_region']
        
        return {
            'feature_index': splited_infos['feature_index'],
            'tau': splited_infos['tau'],
            'left': self.recursive_growth(splited_infos, current_depth+1, X.loc[l_idx], t.loc[l_idx]),
            'right': self.recursive_growth(splited_infos, current_depth+1, X.loc[r_idx], t.loc[r_idx])
        }
    
    def get_split(self, X: pd.DataFrame, y: pd.Series, gini_index: float):
        """
        Given a dataset (full or partial), splits it on the feature of that minimizes the
        sum of squared error
        
        Parameters
        ----------
        X : pd.DataFrame
            features 
        y : pd.Series
            labels
        gini_index: float
            This function is used 
        
        Returns
        -------
        decision : dictionary
            keys are:
            * 'feature_index' -> an integer that indicates the feature (column) of `X`
            on which the data is split
            * 'tau' -> the threshold used to make the split
            * 'left_region' -> array of indices where the `feature_index`th feature of X is lower than `tau`
            * 'right_region' -> indices not in `low_region`
        """
        best_tau_separations = []
        # List containing the best tau and gini_index registered for each feature, follows this structure

        for feature in X.columns:  # Going through every feature
            feature_information_gains = []
            # List containing all registered SSE for this features (It follows this structure:)
            # [ (tau_1, sse_1), (tau_2, sse_2), ..., (tau_j, sse_j) ]
            # Where 'tau_i' is the ith element of the feature's column and 'sse_i' is its registered SSE
            unique_features = pd.unique(X[feature])

            for tau in unique_features:  # Using each value of the feature as tau (For performance reasons)
                # First I get the node division based on the tau and the feature specified
                l_feature, r_feature = self.split_region(X, feature, tau)
                # I initialize my weighted gini index
                tau_weighted_gini_index = 0

                # I store the divisions on variables for organization purposes
                y_right = y[r_feature]
                y_left = y[l_feature]

                # I get the gini index for the left node and the right node
                gini_left = self.gini_index(y_left)
                gini_right = self.gini_index(y_right)

                # For organization purposes I store the amount of elements in the parent node, its
                # left division and right division
                amount_total = len(y)
                amount_right = len(y_right)
                amount_left = len(y_left)

                # Now I sum up the weighted gini index as the definition says
                tau_weighted_gini_index += (amount_left/amount_total)*gini_left
                tau_weighted_gini_index += (amount_right/amount_total)*gini_right

                # Now I calculate the information gain for this specific feature and tau
                tau_information_gain = gini_index - tau_weighted_gini_index
                
                feature_information_gains.append((tau, tau_information_gain, gini_left + gini_right))

            maximum_tau_information_gain = max(feature_information_gains, key=lambda f_s: f_s[1])
            # Getting the maximum information gain
            best_tau_separations.append((feature, *maximum_tau_information_gain))
            # Store the best tau separation in a list for later get the best option
        
        best_separation_criterion = max(best_tau_separations, key=lambda s_c: s_c[2])
        # Now I get the best information gain based on the tau and the feature

        l, r = self.split_region(X, best_separation_criterion[0], best_separation_criterion[1])

        return {
            'feature_index': best_separation_criterion[0],
            'tau': best_separation_criterion[1],
            'left_region': l,
            'right_region': r,
            'gini_index': best_separation_criterion[3],
            'information_gain': best_separation_criterion[2]
        }
    
    def split_region(self, region: pd.Series | pd.DataFrame, feature_index: int, tau: float):
        """
        Given a region, splits it based on the feature indicated by
        `feature_index`, the region will be split in two, where
        one side will contain all points with the feature with values 
        lower than `tau`, and the other split will contain the 
        remaining datapoints.
        
        Parameters
        ----------
        region : pd.Series | pd.DataFrame
            a partition of the dataset (or the full dataset) to be split
        feature_index : int
            the index of the feature (column of the region array) used to make this partition
        tau : float
            The threshold used to make this partition
            
        Return
        ------
        left_partition : pd.Series | pd.DataFrame
            indices of the datapoints in `region` where feature < `tau`
        right_partition : pd.Series | pd.DataFrame
            indices of the datapoints in `region` where feature >= `tau` 
        """
        left_partition = region[region[feature_index] < tau][feature_index].index
        right_partition = region[region[feature_index] >= tau][feature_index].index

        return left_partition, right_partition
    
    def gini_index(self, region: pd.Series) -> float:
        """
        Implements the gini index criterion in a region
        
        Parameters
        ----------
        region : pd.Series
            Array of shape (N,) containing the values of the target values 
            for N datapoints in the training set.
        
        Returns
        -------
        float
            The sum of squared error
            
        Note
        ----
        The error for an empty region should be infinity (use: float("inf"))
        This avoids creating empty regions
        """
        if len(region) == 0:
            return 0
        if isinstance(region, pd.Series):
            region = region.to_numpy()
        
        classes = np.unique(region)
        Gm = 0
        
        for k in classes:
            p_mk = len(region[region == k]) / len(region)
            Gm += p_mk * (1 - p_mk)

        return Gm
    
    def predict_sample(self, sample: pd.Series, node: dict):
        """
        Makes a prediction based on the decision tree defined by `node`
        
        Parameters
        ----------
        node : dictionary
            A node created one of the methods above
        sample : array of size (n_features,)
            a sample datapoint
        """
        if 'value' in node.keys():
            return node['value']
        else:
            if sample[node['feature_index']] < node['tau']:
                return self.predict_sample(sample, node['left'])
            else:
                return self.predict_sample(sample, node['right'])

    def predict(self, X,):
        """
        Makes a prediction based on the decision tree defined by `node`
        
        Parameters
        ----------
        X : array of size (n_samples, n_features)
            n_samples predictions will be made
        """
        predicted_values = pd.Series(np.array([x for x in range(len(X))]))
        predicted_values = predicted_values.apply(lambda i: self.predict_sample(X.iloc[i], self.root))
        return predicted_values

In [21]:
tree = DecisionTree()
tree.fit(X_train, t_train)

In [22]:
def get_precision(t: pd.Series, y: pd.Series) -> float:
    """Gets the precision using the passed parameters

    Args
    ----------
    t: pd.Series
        True labels
    y: pd.Series
        Predictions
    
    Returns
    ----------
    float: The precision of the given prediction
    """
    y = y.to_numpy()
    t = t.to_numpy()
    TP = len(y[
        (y == 1) & (t == 1  )
    ])
    FP = len(y[
        (y == 1) & (t == 0)
    ])
    if TP + FP == 0:
        return 0
    precision = TP / (TP + FP)
    return precision

In [23]:
def print_tree(node, depth):
    if 'value' in node.keys():
        print('.  '*(depth-1), f"[{node['value']}]")
    else:
        print('.  '*depth, f'X_{node["feature_index"]} < {node["tau"]}')
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)

In [24]:
y_test = tree.predict(X_test)

get_precision(t_test, y_test)

1.0

## Random_Forest
## Part 2 - Random Forest for Classification Networks (value: 30%)

In this exercise, you will expand on the previous exercise and implement Random Forests. You will build an ensemble of decision trees and use them for the same classification task from Part 1. The dataset used for this exercise will be the same as in the previous exercise. Your task is to write code to construct a Random Forest model, evaluate its performance, and compare it to the decision tree implementation. 

To complete this exercise, you will write code to implement Random Forest for this problem: 

1. Dataset Splitting: use the same splits you used for Part 1.
2. Implement a function to learn Random Forest – the main steps are detailed below:
	- Initialize an empty Random Forest.
	- Determine the number of decision trees to include in the forest (e.g. 20), and the number of the random features to consider, generally `num_features` $≈\sqrt{p}$ where $p$ is the total number of features.
	- Implement a loop to build the specified number of decision trees:
        - Generate a bootstrap sample from the training set (sampling with replacement).
        - Build a decision tree using the bootstrap sample, using your implementation from Part I.
        - Add the constructed decision tree to the Random Forest.
3. Implement a classification function. Implement a function to classify new instances using the Random Forest:
	- For each instance, pass it through all decision trees in the Random Forest and collect the predictions. Note that you should binarize the prediction of each decision tree, that is, use a threshold of 0.5 to determine the actual class label.
	- The prediction for the random forest will be the frequency of the positive class in the predictions collected by all the decision trees.
4. Run your algorithm and evaluate its performance:
	- Call the function to learn the Random Forest with your training set. You will vary the different parameters of the Random Forest to observe their effect on the performance on the validation set. You will use the training set to learn the tree and the validation set using the Area Under the Roc Curve (AUROC) to find the optimal parameters. Again, keep your trees shallow and don’t build many decision trees, as this could delay the training time quite a lot.
	- Build a Random Forest using the training + validation sets with the best combination of parameters.
	- Classify the instances of the testing set using the Random Forest, calculate the accuracy (threshold: 0.5) and Area Under the ROC Curve (AUROC) and report the results.
5. Experimentation: Compare the performance of Random Forests with the single decision tree implementation from the previous exercise reporting the performance on the test set in a table (either a dataframe or markdown). 


In [11]:
## your code goes here:

## K-Means
## Part 3 – Clustering with K-means (value: 40%)

In this exercise, you will explore clustering by implementing the K-means algorithm. You will write code to perform K-means clustering while visualizing the movement of the centroids at each iteration. 

To complete this exercise, you will write code to implement K-means for clustering: 

1. Dataset Preparation: Run the cells provided in the notebook that generate the artificial data points for this exercise.
2. K-means Clustering:
	- Initialize K cluster centroids by selecting K points from your dataset at random.
	- Implement a loop to perform the following steps until convergence (or until a specified maximum number of iterations is reached, e.g., 150):
        - Assign each data point to the nearest centroid (you will have to calculate the Euclidean distance between the data point and each centroid).
        - Update each centroid by moving it to the mean of all data points assigned to it.
        - Check for convergence by comparing the new centroids with the previous centroids. If the difference is smaller than an $\epsilon=1^{-4}$, exit the loop.
3. Centroid Movement Visualization:
	- At 5 different moments during training, plot a figure showing the centroids and the points. Figure 1 should show the situation at the beginning, before learning. Figure 5 should show the situation at the end of the learning. The remaining Figures 2-4 should show intermediary situations.
	- For each figure, each centroid will be represented by a large black cross and each cluster with a different colour, the points must be coloured according to their respective cluster.
4. Sum of squared distances:
	- Along with plotting the centroid movement, calculate the sum of squared distances at each iteration as follows:
        - $\sum_{j=1}^K \sum_{n \in S_j}d(x_n,\mu_j )^2$, where $K$ is the number of clusters, $x_n$ represents the $n^{th}$ datapoint, $n \in S_j$ indicates a set of points that belong to cluster $S_j$, $\mu_j$ is the mean of the datapoints in $S_j$ and $d(x_n,\mu_j)$ indicates the Euclidean distance between $x_n$ and $\mu_j$.
	- Make a plot of the sum of squared distances at each iteration. 


In [12]:
# Generate artificial data points
np.random.seed(13)
num_samples = 200
num_features = 2
X = np.random.randn(num_samples, num_features) * 1.5 + np.array([[2, 2]])
X = np.concatenate([X, np.random.randn(num_samples, num_features) * 3 + np.array([[-5, -5]])])
X = np.concatenate([X, np.random.randn(num_samples, num_features) * 2 + np.array([[7, -5]])])

In [13]:
## your code goes here: